In [316]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

# Figure 2.1: Breast cancer incidence

In [317]:
df_incidence_mortality = pd.read_csv('dataset-asr-inc-and-mort-females-in-2022-world.csv')
#df_incidence_mortality = df_incidence_mortality[df_incidence_mortality["Label"].notna()]
df_incidence_mortality = df_incidence_mortality[["Label", "Type", "ASR (World)", "Crude rate", "Total"]]
df_incidence_mortality

,Label,Type,ASR (World),Crude rate,Total
0,"Lip, oral cavity",0,2.29,3.09,120847
1,"Lip, oral cavity",1,1.07,1.47,57630
2,Salivary glands,0,0.49,0.62,24120
3,Salivary glands,1,0.18,0.25,9953
4,Oropharynx,0,0.39,0.51,20061
...,...,...,...,...,...
77,NaN,1,0.00,0.00,0
78,NaN,1,0.00,0.00,0
79,NaN,1,0.00,0.00,0
80,NaN,1,0.00,0.00,0


In [318]:
df_incidence = df_incidence_mortality[df_incidence_mortality["Type"] == 0]
df_incidence.index = df_incidence["Label"]
df_incidence = df_incidence.drop(columns=["Label", "Type"])

df_mortality = df_incidence_mortality[df_incidence_mortality["Type"] == 1]
df_mortality.index = df_mortality["Label"]
df_mortality = df_mortality.drop(columns=["Label", "Type"])

In [319]:
df_asr = pd.concat([df_incidence["ASR (World)"], df_mortality["ASR (World)"]], axis=1)
df_asr.columns = ["Incidence", "Mortality"]

df_asr = df_asr.sort_values(by="Incidence", ascending=False)

In [320]:
df_total = pd.concat([df_incidence["Total"], df_mortality["Total"]], axis=1)
df_total.columns = ["Incidence", "Mortality"]

df_total = df_total.sort_values(by="Incidence", ascending=False)

In [321]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)

# Increase space between subplots
plt.subplots_adjust(wspace=0.8)

In [322]:
df_barplot = df_asr.head(6).copy()

df_barplot["Incidence"] = -df_barplot["Incidence"]
df_barplot.head()

sns.barplot(data=df_barplot, y=df_barplot.index, x="Incidence", color="blue", label="Incidence", ax=ax1)
sns.barplot(data=df_barplot, y=df_barplot.index, x="Mortality", color="red", label="Mortality", ax=ax1)

# Add bar labels, use absolute values
ax1.bar_label(ax1.containers[0], labels=np.abs(df_barplot["Incidence"]))
ax1.bar_label(ax1.containers[1], labels=np.abs(df_barplot["Mortality"]))

# Add space between bars
delta = 20
ax1.set_xlim(min(df_barplot["Incidence"]) - delta, max(df_barplot["Mortality"]) + delta)

# Set axis labels
ax1.set_ylabel("Cancer Type")
ax1.set_xlabel("Age-standardized rate (World)")

# Set title
ax1.set_title("a: Incidence and Mortality of Cancer Types")

# Use absolute values for x-axis
ax1.set_xticklabels(np.abs(ax1.get_xticks()).astype(int))

/tmp/ipykernel_213675/3079502842.py:25: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(np.abs(ax1.get_xticks()).astype(int))


[Text(-80.0, 0, '80'),
 Text(-60.0, 0, '60'),
 Text(-40.0, 0, '40'),
 Text(-20.0, 0, '20'),
 Text(0.0, 0, '0'),
 Text(20.0, 0, '20'),
 Text(40.0, 0, '40')]

In [323]:
# Add pie chart
df_pieplot = df_asr.copy()
df_pieplot["Incidence"] = df_pieplot["Incidence"] / df_pieplot["Incidence"].sum()
df_pieplot["Mortality"] = df_pieplot["Mortality"] / df_pieplot["Mortality"].sum()

df_pieplot = df_pieplot.head(6)

# Create "others" category
df_pieplot.loc["Others"] = 1 - df_pieplot.sum()

ax2.pie(df_pieplot["Incidence"], labels=df_pieplot.index, autopct='%1.1f%%', startangle=90)

# Set title
ax2.set_title("b: Relative Incidence of Cancer Types")

Text(0.5, 1.0, 'b: Relative Incidence of Cancer Types')

In [324]:
# Increase title size
fig.suptitle("Global Incidence and Mortality rates of Cancer Types in women in 2022", fontsize=16)

Text(0.5, 0.98, 'Global Incidence and Mortality rates of Cancer Types in women in 2022')

In [325]:
plt.tight_layout()
plt.savefig("../chapters/2_background/figures/incidence.png")